---
format:
    html:
        embed-resources: true
---

# Cleaning: Part-2 

The goal here is exactly the same as `2-cleaning-1.ipynb`, except this time we should repeat the exercise but by leveraging LLM APIs and prompt engineering to stream line the cleaning process. 

Essentially, our job is to write an LLM wrapper to clean the job descriptions. 


In [2]:
import pandas as pd
import numpy as np

## API

In [3]:
import google.generativeai as genai

In [4]:
import json
with open('C:/Users/admin/.google-api.json') as f:
    keys = json.load(f)

In [5]:
API_KEY = keys['googleapi']

In [6]:
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)

## How AI Works: A Simplified Explanation

Artificial intelligence (AI) is a vast and complex field, but at its core, it aims to create intelligent agents, systems that can reason, learn, and act autonomously. Here's a simplified explanation of how AI works:

**1. Data is the fuel:** AI systems learn from data. They need vast amounts of data to identify patterns, trends, and relationships. This data can be anything from images and text to sensor readings and financial records.

**2. Algorithms are the engine:** Algorithms are sets of instructions that define how the AI system processes data and makes decisions. These algorithms are inspired by human cognitive processes like learning, problem-solving, and decision-making.

**3. Machine learning: the core of modern AI:**  Machine learning is a subset of AI that focuses on developing algorithms that allow computers to learn from data without being explicitly programmed. These algorithms can be broadly classified into:

* **Supervised lear

## Prompt

In [7]:
import datetime

In [ ]:
# Prompt

def generate_cleaning_prompt(description, highlights, extensions):
    """Generates a prompt for cleaning the job description and highlights, outputting plain text."""
    return f"""
    You will receive a job description (description), job highlights (highlights), and extensions (extensions). Please extract the following fields and provide the output as plain text. Each field should have the same format across different job descriptions.

    **Base Date**: The reference date is 10/26. Use this as the basis for calculating any date fields like "posted X days ago".

    **Industry Requirement**: Each company should be assigned a single industry. Choose from the following standardized list:
    - Technology
    - Finance
    - Healthcare
    - Retail
    - Manufacturing
    - Energy
    - Telecommunications
    - Transportation
    - Education
    - Hospitality

    Fields to extract:
    - Job Title
    - Company Name
    - Sector/Industry(choose a single industry from the provided list)
    - Location
    - Job Type
    - Salary
    - Annual Average Salary(caculate based on Salary, must be a numeric value)
    - Experience Level
    - Education Requirements
    - Skills/Technologies Required
    - Job Responsibilities/Duties
    - Required Years of Experience(must be a numeric value)
    - Benefits
    - Remote Work Options
    - Application Deadline(present in mm/dd/yyyy format)
    - Job Posting Date (based on "posted X days ago" from 10/26,present in mm/dd/yyyy format)
    - Job Description Length(must be a numeric value)
    - Keywords/Frequency of Terms
    - Certifications Required or Preferred
    - Team Size
    - Company Size
    - Company Reputation/Ranking
    - Job Posting Platform
    - Company Values or Culture
    - Visa Sponsorship Availability
    - Interview Process Information
    - Expected Start Date(present in mm/dd/yyyy format)
    - Job Posting Expiry Date(present in mm/dd/yyyy format)
    - Gender Diversity Language
    - Working Hours/Shift Type
    - Required Language Skills(If not mentioned, return English as a default language skill)
    - Job Location Proximity to Major Cities
    - Travel Requirements
    - Team Collaboration Tools Mentioned
    - Reporting Line
    - Job Benefits Related to Learning & Development
    - Company Stock Options
    - Required Soft Skills
    - Company Perks

    Example format:
    Job Title: [Job Title]
    Company Name: [Company Name]
    Sector/Industry: [Sector/Industry]
    Location: [Location]
    Job Type: [Job Type]
    Salary: [Salary]
    Annual Average Salary: [Annual Average Salary]
    Experience Level: [Experience Level]
    Education Requirements: [Education Requirements]
    Skills/Technologies Required: [Skills/Technologies]
    Job Responsibilities/Duties: [Responsibilities]
    Required Years of Experience: [Years of Experience]
    Benefits: [Benefits]
    Remote Work Options: [Remote Options]
    Application Deadline: [Deadline]
    Job Posting Date: [Posting Date]
    Job Description Length: [Description Length]
    Keywords/Frequency of Terms: [Keywords]
    Certifications Required or Preferred: [Certifications]
    Team Size: [Team Size]
    Company Size: [Company Size]
    Company Reputation/Ranking: [Reputation]
    Job Posting Platform: [Platform]
    Company Values or Culture: [Values]
    Visa Sponsorship Availability: [Sponsorship]
    Interview Process Information: [Interview Info]
    Expected Start Date: [Start Date]
    Job Posting Expiry Date: [Expiry Date]
    Gender Diversity Language: [Diversity Language]
    Working Hours/Shift Type: [Shift Type]
    Required Language Skills: [Language Skills]
    Job Location Proximity to Major Cities: [Proximity]
    Travel Requirements: [Travel]
    Team Collaboration Tools Mentioned: [Tools]
    Reporting Line: [Reporting Line]
    Job Benefits Related to Learning & Development: [Learning Benefits]
    Company Stock Options: [Stock Options]
    Required Soft Skills: [Soft Skills]
    Company Perks: [Perks]

    **Job Description**: {description}
    **Job Highlights**: {highlights}
    **Extensions**: {extensions}

    Please ensure each field follows the format exactly as shown above, with values filled in after each colon.
    
    All the things that not specified return NA.
    """

In [9]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [ ]:
#Clean data from json

import os
import time
import json

def fetch_job_data_from_json(folder_path, output_file, log_file="progress.log"):
    """Extracts job data from JSON files in a folder and saves it to a specified text file."""
    
    # Load the progress log if it exists
    processed_jobs = set()
    if os.path.exists(log_file):
        with open(log_file, "r") as log:
            processed_jobs = set(line.strip() for line in log)
    
    with open(output_file, "a", encoding="utf-8") as f:  # Append mode
        for filename in os.listdir(folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(folder_path, filename)
                
                with open(file_path, "r", encoding="utf-8") as json_file:
                    data = json.load(json_file)
                    for job_index, job in enumerate(data):
                        job_id = f"{filename}-{job_index}"  # Create a unique ID for each job
                        if job_id in processed_jobs:
                            continue  # Skip already processed jobs
                        
                        description = job.get("description", "")
                        highlights = job.get("job_highlights", "")
                        extensions = job.get("extensions", "")
                        
                        # Generate the prompt for data extraction
                        prompt = generate_cleaning_prompt(description, highlights, extensions)
                        success = False
                        retries = 3  # Number of retries for quota errors
                        wait_time = 60  # Wait time in seconds before retrying
                        
                        for attempt in range(retries):
                            try:
                                # Call the Gemini API to retrieve generated text content
                                model = genai.GenerativeModel("gemini-1.5-flash")
                                response = model.generate_content(
                                    prompt,
                                    generation_config=genai.types.GenerationConfig(
                                        candidate_count=1,
                                        max_output_tokens=1000,
                                        temperature=0.7
                                    )
                                )
                                
                                # Attempt to extract the content part and write it to the output file
                                if response.candidates and response.candidates[0].content.parts:
                                    generated_text = response.candidates[0].content.parts[0].text.strip()
                                    f.write(generated_text + "\n\n")
                                    
                                    # Record the job as processed
                                    with open(log_file, "a") as log:
                                        log.write(job_id + "\n")
                                    
                                    success = True  # Mark as successful if no exceptions occurred
                                    break
                                else:
                                    raise IndexError("Empty candidates or content parts in response")
                                    
                            except IndexError as e:
                                print(f"IndexError: {e}. Retrying in {wait_time} seconds... (Attempt {attempt + 1} of {retries})")
                                time.sleep(wait_time)
                            except Exception as e:
                                error_message = str(e).lower()
                                if "quota exceeded" in error_message or "rate limit" in error_message:
                                    print(f"API quota exceeded, waiting {wait_time} seconds before retrying... (Attempt {attempt + 1} of {retries})")
                                    time.sleep(wait_time)
                                else:
                                    print(f"Unexpected error processing job in file {filename}: {e}")
                                    break

                        if not success:
                            print(f"Failed to process job in file {filename} after {retries} attempts.")

In [ ]:
# define folder path
json_folder_path = "./data"

In [14]:
output_text_file = "cleaned_job_data.txt"

In [ ]:
# Parse the text generated from api

def parse_cleaned_data(file_path):
    """Parses the cleaned text data and creates a structured DataFrame."""
    jobs_data = []
    with open(file_path, "r", encoding="utf-8") as f:
        job_info = {}
        for line in f:
            if line.strip() == "":
                if job_info:
                    jobs_data.append(job_info)
                    job_info = {}
                continue
            if ":" in line:
                key, value = line.split(":", 1)
                job_info[key.strip()] = value.strip()
                
        if job_info:  # Append the last job entry
            jobs_data.append(job_info)

    # Convert the list of job data dictionaries to a DataFrame
    df = pd.DataFrame(jobs_data)
    return df

In [16]:
# Generate txt
fetch_job_data_from_json(json_folder_path, output_text_file)

In [ ]:
cleaned_df = parse_cleaned_data(output_text_file)

In [27]:
cleaned_df.head()

,Job Title,Company Name,Sector/Industry,Location,Job Type,Salary,Annual Average Salary,Experience Level,Education Requirements,Skills/Technologies Required,...,Working Hours/Shift Type,Required Language Skills,Job Location Proximity to Major Cities,Travel Requirements,Team Collaboration Tools Mentioned,Reporting Line,Job Benefits Related to Learning & Development,Company Stock Options,Required Soft Skills,Company Perks
0,AI Solution Architect –GPU,AMD,Technology,"San Jose, CA",Full-time,NaN,NaN,NaN,BS required. MS preferred,"GPU, CUDA, ROCm, TensorRT, vLLM, Megatron-LM, ...",...,NaN,English,NaN,NaN,NaN,NaN,NaN,Many AMD employees have the opportunity to own...,Excellent written and verbal communication ski...,NaN
1,AI Architect,American Airlines,Transportation,"Dallas, TX",Full-time,NaN,NaN,Senior,"Bachelor's degree in Computer Science, Compute...","Enterprise/Solutions Architecture, software en...",...,NaN,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Travel perks, health benefits, wellness progra..."
2,Solutions Architect,Amivero,Technology,NaN,Full-time,NaN,NaN,Senior,"Bachelor's in Computer Science, Data Science, ...","Python, R, Java, C++, TensorFlow, PyTorch, sci...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AI/GenAI Solution Architect,NaN,Technology,NaN,Full-time,USD 40k - USD 200k,120000,Experienced,"Bachelor's degree in Computer Science, Enginee...","Azure, Azure Machine Learning, Azure Cognitive...",...,NaN,English,NaN,NaN,NaN,NaN,NaN,NaN,"Problem-solving, Communication",NaN
4,Senior Solutions Architect,NaN,Technology,NaN,Full-time,NaN,NaN,Senior,Bachelor's Degree in Computer Science,"DevOps, cloud/hybrid environments, unstructure...",...,NaN,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 39 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Job Title                                       849 non-null    object 
 1   Company Name                                    759 non-null    object 
 2   Sector/Industry                                 867 non-null    object 
 3   Location                                        375 non-null    object 
 4   Job Type                                        874 non-null    object 
 5   Salary                                          503 non-null    object 
 6   Annual Average Salary                           465 non-null    object 
 7   Experience Level                                678 non-null    object 
 8   Education Requirements                          764 non-null    object 
 9   Skills/Technologies Required               

In [ ]:
# Drop duplicates
duplicate_rows = cleaned_df.duplicated(subset=['Job Title', 'Company Name', 'Location'])

In [36]:
cleaned_df.drop_duplicates(subset=['Job Title', 'Company Name', 'Location'], inplace=True)

In [ ]:
# Drop na for some vairables
print(cleaned_df.isnull().sum())

Job Title                                          11
Company Name                                       85
Sector/Industry                                     3
Location                                          430
Job Type                                            1
Salary                                            328
Annual Average Salary                             357
Experience Level                                  169
Education Requirements                             96
Skills/Technologies Required                       27
Job Responsibilities/Duties                        39
Required Years of Experience                      198
Benefits                                          235
Remote Work Options                               472
Application Deadline                              741
Job Posting Date                                    4
Job Description Length                             15
Keywords/Frequency of Terms                       142
Certifications Required or P

In [39]:
df_cleaned = cleaned_df.dropna(subset=['Annual Average Salary'])

In [42]:
df_cleaned = df_cleaned.dropna(subset=['Job Title'])

In [43]:
df_cleaned = df_cleaned.dropna(subset=['Company Name'])

In [45]:
df_cleaned = df_cleaned.dropna(subset=['Salary'])

In [46]:
print(df_cleaned.isnull().sum())

Job Title                                           0
Company Name                                        0
Sector/Industry                                     1
Location                                          213
Job Type                                            0
Salary                                              0
Annual Average Salary                               0
Experience Level                                   65
Education Requirements                             41
Skills/Technologies Required                        9
Job Responsibilities/Duties                        15
Required Years of Experience                       72
Benefits                                           38
Remote Work Options                               229
Application Deadline                              356
Job Posting Date                                    3
Job Description Length                              7
Keywords/Frequency of Terms                        84
Certifications Required or P

In [49]:
# Ensure some columns is numeric
numeric_columns = ['Annual Average Salary', 'Required Years of Experience', 'Team Size', 'Company Size']
for col in numeric_columns:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')  # Convert to numeric, handle errors as NaN


# Ensure Dates are in datetime format
date_columns = ['Job Posting Date', 'Application Deadline','Expected Start Date','Job Posting Expiry Date']  # replace with your date columns
for col in date_columns:
    df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')  # Convert to datetime, set errors to NaT

In [50]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 5 to 874
Data columns (total 39 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Job Title                                       396 non-null    object        
 1   Company Name                                    396 non-null    object        
 2   Sector/Industry                                 395 non-null    object        
 3   Location                                        183 non-null    object        
 4   Job Type                                        396 non-null    object        
 5   Salary                                          396 non-null    object        
 6   Annual Average Salary                           390 non-null    float64       
 7   Experience Level                                331 non-null    object        
 8   Education Requirements                          355 non

In [ ]:
# Clean outliers
from scipy.stats import zscore

# Calculate Z-scores for Salary
df_cleaned['salary_zscore'] = zscore(df_cleaned['Annual Average Salary'].dropna())
salary_z_outliers = df_cleaned[abs(df_cleaned['salary_zscore']) > 3]

# Calculate Z-scores for Required Years of Experience
df_cleaned['experience_zscore'] = zscore(df_cleaned['Required Years of Experience'].dropna())
experience_z_outliers = df_cleaned[abs(df_cleaned['experience_zscore']) > 3]

# Calculate Z-scores for Job Description Length
df_cleaned['description_zscore'] = zscore(df_cleaned['Job Description Length'].dropna())
description_z_outliers = df_cleaned[abs(df_cleaned['description_zscore']) > 3]

# Display Z-score outliers
print("Salary Z-Score Outliers:\n", salary_z_outliers[['Job Title', 'Annual Average Salary', 'salary_zscore']])
print("Experience Z-Score Outliers:\n", experience_z_outliers[['Job Title', 'Required Years of Experience', 'experience_zscore']])
print("Description Length Z-Score Outliers:\n", description_z_outliers[['Job Title', 'Job Description Length', 'description_zscore']])

Salary Z-Score Outliers:
                                          Job Title  Annual Average Salary  \
64                             AI Product Engineer               364400.0   
243  Principal Engineer, Cloud ML Compute Services               338500.0   
632                              ML Infra Engineer               362500.0   

     salary_zscore  
64        3.700036  
243       3.275369  
632       3.668883  
Experience Z-Score Outliers:
                          Job Title  Required Years of Experience  \
357  Data Analyst (Data Scientist)                          52.0   

     experience_zscore  
357          11.709498  
Description Length Z-Score Outliers:
                                          Job Title  Job Description Length  \
357                  Data Analyst (Data Scientist)                  1716.0   
707        Supervisory Operations Research Analyst                  2210.0   
711  Operational Research Analyst (Data Scientist)                  1384.0   

     descript

In [54]:
# Drop outliers
z_threshold = 3

df_cleaned = df_cleaned[~((df_cleaned['salary_zscore'].abs() > z_threshold) & (df_cleaned['salary_zscore'].notna()))]

df_cleaned = df_cleaned[~((df_cleaned['experience_zscore'].abs() > z_threshold) & (df_cleaned['experience_zscore'].notna()))]

df_cleaned = df_cleaned[~((df_cleaned['description_zscore'].abs() > z_threshold) & (df_cleaned['description_zscore'].notna()))]

df_cleaned.reset_index(drop=True, inplace=True)

In [55]:
cleaned_df.to_csv("cleaned_jobs_data.csv", index=False, encoding="utf-8")